# O Problema

Temos no CEFET MG o grupo Grupo de Computação Competitiva (GCC) que tem como uma de suas atividades a participação na maratona de programação.

A maratona de programação é uma competição onde equipes competem entre si em um período de tempo pré determinado a tentar resolver o maior número de problema em menos tempo.

O objetivo deste trabalho, é desenvolver um algoritmo de recomendação que ajude no treinamento dos competidores para a preparação para a maratona de programação.

Atualmente, o treinamento funciona da seguinte maneira. Existem alguns sites, chamados de juízes online, que funcionam como um repositório de problemas. O competidor então consegue ler o problema, desenvolver uma solução e submete-la para o juiz. O juiz então dará a resposta se a solução foi aceita ou não.

O problema do treinamento atual é que existem milhares de problemas a serem resolvidos. Isso faz com que o competidor perca tempo em problemas que não acrescentem em conteúdo para o seu desenvolvimento, ou então, escolha problemas muito difíceis para serem resolvidos, o que torna seu treinamento desmotivador.

# Introdução

Neste capítulo, será descrito em detalhes, a fase de coleta de dados, definição e treinamento do modelo e implementação da solução proposta.



# Dataset

## A fonte

A primeira decisão a ser tomada foi na escolha de qual juiz online utilizar como fonte de informações. Algumas considerações foram feitas a fim de decidir o juiz, dentre elas:

 - O juiz a ser escolhido deve ser frequentemente utilizado pelos alunos do GCC, uma vez que são os principais usuários do sistema.
 - O juiz deve possuir informações relevantes sobre os problemas em seu repositório, como dificuldade e categoria que será utilizado posteriormente como forma de validação do modelo.
 - O juiz deve possuir um número quantitativamente relevante para a coleta.
 - O juiz deve permitir a coleta das informações.
 
Os critérios definidos levaram a escolha do URI Online Judge. O URI é o juiz com mais problemas resolvidos pelos alunos do GCC, como pode ser visto no BANG. Seus problemas possuem nível de dificuldade, categoria, número de usuários únicos que o resolveram e em alguns problemas, é possível obter até o assunto que o problema aborda dentro daquela categoria. No ano de 2017, o URI atingiu a marca de 1656 problemas, 195329 usuários de 170 países e 7956566 submissões, como pode ser visto em (https://www.urionlinejudge.com.br/year-in-numbers/2017/2/). Todas essas informações são públicas e podem ser vistas no próprios site.

## As informações

Com o juiz definido, o próximo passo foi definir o que e o quanto de informações seriam adquiridas do juiz.

No URI, cada usuário possui uma página de perfil que informa quando e quais problemas o usuário resolveu. Cada registro dessa página é conhecido como **solução**. Além dos perfis de usuários, existe também a página de problemas, onde é possível encontrar informações detalhadas de cada problema, como categoria, dificuldade e número usuários que conseguiram resolver o problema.

Dessa forma, através das informações dos usuários e a descrição dos problemas, foi possível construir um dataset composto por um conjunto de soluções, onde cada solução é composta por:

 - **user_id**: usuário que resolveu o problema
 - **problem_id**: problema que foi resolvido
 - **date**: data de quando o problema foi resolvido
 - **category_id**: categoria do problema resolvido
 - **level**: dificuldade do problema resolvido
 - **solved**: quantidade total de usuários únicos do juiz que resolveram o problema
 - **name**: nome do problema
 - **topics**: assuntos relacionados ao conteúdo do problema
 
Foi então coletado, as informações de todos os usuários que haviam resolvido pelo menos 300 problemas, cerca de 20% do número total de problemas disponíveis no juiz. A coleta resultou em um conjutno de soluções com 618 usuários e 218619 soluções.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy import spatial
from datetime import datetime
from pprint import pprint
from IPython.display import clear_output
from numpy.random import rand

Configurações

In [2]:
# 0 - All
# 1 - Beginner
# 2 - Ad-Hoc
# 3 - String
# 4 - Libraries
# 5 - Math
# 6 - Paradigms
# 7 - Graph
# 8 - Geometry

CATEGORY = 7
NR_FEATURES = 1
NR_EPOCHS = 1
NR_ITERATIONS = 1

In [3]:
def read_dataset():
    solutions_df = pd.read_csv('solutions_uva.csv')
    if CATEGORY:
        solutions_df = solutions_df.loc[solutions_df['category_id'] == CATEGORY]

    return solutions_df

In [4]:
solutions_df = read_dataset()
solutions_df[:5]

,user_id,problem_id,date,category_id,solved,name,starred
0,3719,260,00/02/21 07:08,7,3361,Il Gioco dell'X,False
3,3719,383,00/02/21 07:48,7,2472,Shipping Routes,False
10,3719,200,00/03/09 08:49,7,5367,Rare Order,False
13,3719,532,00/03/09 10:08,7,5181,Dungeon Master,False
17,3719,247,00/03/19 01:32,7,2650,Calling Circles,True


## Filtro

De forma a otimizar o tempo  de execução do algoritmo na fase de desenvolvimento, foram utilizadas soluções referentes a uma única categoria.

Quantidade de soluções do dataset:

In [5]:
len(solutions_df)

48428

Quantidade de problemas do dataset:

In [6]:
solutions_df['problem_id'].nunique()

95

Quantidade de usuários do dataset:

In [7]:
solutions_df['user_id'].nunique()

1998

Quantidade de categorias do dataset:

In [8]:
solutions_df['category_id'].nunique()

1

# Modelo

## Considerações iniciais

obs: cada exemplo poderia conter o link de um problema no URI

obs2: http://codeforces.com/blog/entry/47442

O aprendizado é um processo gradativo, por se realizar por meio de operações crescentemente complexas. A cada aprendizagem novos elementos são acrescidos às experiências anteriores em dimensão gradativa e ascendente. Também se caracteriza por ser um processo cumulativo, visto que a experiência de aprendizagem atual utiliza-se das experiências anteriores. (http://producao.virtual.ufpb.br/books/edusantana/fundamentos-psicologicos-da-educacao-livro/livro/livro.chunked/ch06s03.html)

Tal como as aulas de cálculo, onde começamos pelo o estudo de funções, passamos por limites e derivadas até finalmente o estudo das integrais. Da mesma forma acontece o treinamento para a maratona de programação. Por exemplo, no estudo de estruturas de dados, iniciamos com o estudo de estruturas de dados simples como pilhas e filas, seguido por estruturas como árvores e grafos até chegarmos em estruturas de dados complexas como a Segment Tree.

Baseado nas informações acima citadas, podemos concluir que um competidor para resolver um problema mais complexo, necessita desenvolver conhecimentos fundamentais. E a partir desta conclusão temos o primeiro problema a ser resolvido por este trabalho: **Identificar os tópicos necessários a se saber para a solução de um problema**

Tendo isso em mente, essa será a primeira tarefa desenvolvida. Para isso iremos utilizar a técnica de Collaborative Filtering.


## Colaborative Filtering

> A Colaborative Filtering é um método para fazer previsões automáticas (filtragem) sobre os interesses de um usuário, coletando preferências ou informações de gosto de muitos usuários (colaborando). O pressuposto subjacente da abordagem de filtragem colaborativa é que se uma pessoa A tiver a mesma opinião que uma pessoa B em um problema, é mais provável que a opinião de B seja diferente em relação a uma pessoa escolhida aleatoriamente. Por exemplo, um sistema de recomendação de filtragem colaborativa para os gostos da televisão poderia fazer previsões sobre o programa de televisão que um usuário deveria gostar, dado uma lista parcial dos gostos desse usuário (gosta ou não gosta).

Imagine uma situação onde temos um dataset composto por avaliações de filmes. Cada avaliação é feita através de uma nota entre 1 e 5. Teriamos uma tabela da seguinte forma:

| Filmes  | Ana | Beto | Carlos | Dani |
|---------|:---:|:----:|:------:|:----:|
| Querido John |  5  |   4  |    0   |   0  |
| Titanic |  3  |   5  |    0   |   0  |
| Batman |  0  |   0  |    4   |   5  |
| Thor |  0  |   0  |    5   |   4  |

As notas dadas aos filmes podem ser relacionadas as preferências do usuário em relação aos gêneros dos filmes. A alta nota de Beto ao filme Titanic pode ser justificada por Beto gostar de filmes de romance.

Logo, se fossem conhecidos as caraterísticas dos filmes e as preferências do usuários, poderiamos prever uma avaliação de um filme por um usuário, e com isso, conseguir sugerir o melhor filme para o usuário.

| Filme/Características | Ação | Romance |
|---------|:--:|:--:|
| Querido John | ? | ? |
| Titanic | ? | ? |
| Batman | ? | ? |
| Thor | ? | ? |


| Usuário/Preferência  | Ação | Romance |
|:--:|:-:|:--:|
| Ana | ? | ? |
| Beto | ? | ? |
| Carlos | ? | ? |
| Dani | ? | ? |


Podemos então representar cada filme com um vetor de **características** $X$ e cada usuário com um vetor de **preferências** $\theta$ ambos de mesma dimensão. A nota final do filme, pode ser então dada pelo produto $X ⋅ \theta^T$

Embora não tenhamos os valores de $X$ e $\theta$ estes valores podem ser obtidos através do nosso dataset seguindo os seguintes passos:

1. Iniciamos $X$ com valores randômicos.
2. A partir daí temos um problema de otimização, onde o nosso objetivo é encontrar os valores de $\theta$ que minimizem a diferença entre a nota do filme $f$ dado pelo usuário $u$ pelo produto $ X_u \cdot \theta_f^T $.
3. Repetimos o procedimento 2, porém com o objetivo de encontrar os novos valores de $X$ utilizando os valores de $\theta$ encontrados. 
4. Repetimos os procedimentos 2 e 3, até encontrarmos valores estáveis de $\theta$ e $X$ ou por um número pré determinado de iterações.

In [9]:
# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
def gradient_descent(x, y, alpha=0.01, nr_iterations=NR_ITERATIONS):

    # m - number of samples
    # n - number features
    m, n = x.shape

    # insert column theta_zero = 1
    x = np.c_[ np.ones(m), x]

    theta = np.ones(n + 1)
    x_transpose = x.transpose()

    for iter in range(0, nr_iterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient

    return theta

In [10]:
users_preferences = [
    [rand(), rand()],
    [rand(), rand()],
    [rand(), rand()],
    [rand(), rand()],
]

movies_characteristics = [
    [None, None],
    [None, None],
    [None, None],
    [None, None],
]

ratings = [
    [5, 4, 0, 0],
    [3, 5, 0, 0],
    [0, 0, 4, 5],
    [0, 0, 5, 4],
]

ratings_t = np.transpose(ratings).tolist()

nr_users = len(users_preferences)
nr_movies = len(movies_characteristics)

i = 0
while i <= 100:
    clear_output(wait=True)
    print('Progress: {}%'.format(i))
    i += 1

    for movie in range(nr_movies):
        x = np.array(
            users_preferences
        )

        y = np.array(
            ratings[movie]
        )

        movies_characteristics[movie] = gradient_descent(x, y)[1:]


    for user in range(nr_users):
        x = np.array(
            movies_characteristics
        )

        y = np.array(
            ratings_t[user]
        )

        users_preferences[user] = gradient_descent(x, y)[1:]

Progress: 100%


Preferências dos usuários

In [11]:
users_preferences

[array([ 0.99022932,  0.99022932]),
 array([ 0.99269872,  0.99269872]),
 array([ 0.9927303,  0.9927303]),
 array([ 0.9927303,  0.9927303])]

Características dos filmes

In [12]:
movies_characteristics

[array([ 0.99269872,  0.99269872]),
 array([ 0.99022932,  0.99022932]),
 array([ 0.9927303,  0.9927303]),
 array([ 0.9927303,  0.9927303])]

Com os valores de $X$ e $\theta$ em mãos, podemos então prever os votos dos usuários através do produto $X ⋅ \theta^T$

In [13]:
np.rint(np.dot(users_preferences, np.transpose(movies_characteristics))) + 2

array([[ 4.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.],
       [ 4.,  4.,  4.,  4.]])

# Adaptação

O conceito acima citado, foi estudado e então adaptado ao nosso problema. Primeiramente foram identificadas as características de cada problema, assim como identificamos as características dos filmes. Então identificamos as características do usuário, e assim então poderemos encontrar o problema mais adequado ao treinamento do usuário.

## As características do problema

No processo de adaptação, as características dos filmes foram definidas como **requisitos** do problema. Ou seja, aquilo que deve ser de conhecimento para que um usuário possa resolver o problema.

Da mesma forma, as preferências do usuário foram definidas com **habilidades**, ou seja, aquilo que é de conhecimento do usuário e o habilita a solucionar problemas que tenha seus requisitos contidos em suas habilidades.

Se um usuário $U_i$ consegue resolver um problema $P_j$, podemos assumir que o usuário $U_i$ tem habilidades suficientes para suprir os requisitos necessários para a solução do problema $P_j$.

A partir dessas premissas iremos identificar quais são os requisitos dos problemas baseado nos histórico de soluções de nosso dataset.

Para isso, iremos realizar o seguinte procedimento para cada um dos problemas em nosso dataset:

Encontrar todos os usuários que resolveram o problema P:

In [14]:
def get_problems(solutions_df):
    problems = set(solutions_df['problem_id'])

    return list(problems)

def get_users_that_solved_the_problem(solutions_df, problem_id):
    solutions_of_the_problem_df = solutions_df[solutions_df['problem_id'] == problem_id]
    users = solutions_of_the_problem_df['user_id']
    
    return list(users)

Para cada usuário, iremos buscar todos os problemas que foram solucionados antes do problema P

In [15]:
def get_date_when_user_solved_the_problem(solutions_df, user_id, problem_id):
    solution = solutions_df[(solutions_df['problem_id'] == problem_id) & (solutions_df['user_id'] == user_id)]
    solution_date = solution['date'].values[0]

    return solution_date

def get_past_problems(solutions_df, user_id, problem_id):
    solution_date = get_date_when_user_solved_the_problem(solutions_df, user_id, problem_id)
    past_solutions = solutions_df[(solutions_df['date'] < solution_date) & (solutions_df['user_id'] == user_id)]
    problems = past_solutions['problem_id']

    return list(problems)

Iremos então criar uma matriz $M$. Onde cada linha corresponde a um problema $P_i$ solucionado antes do problema $P$, e cada coluna corresponde a um usuário $U_j$ que em algum momento resolveu o problema $P$.

Se o problema $P_i$ foi resolvido pelo usuário $U_j$ temos que $M_{i,j} = 1$, caso contrário $M_{i,j} = 0$.

In [16]:
def optimize_matrix(matrix):
    matrix = matrix[~np.all(matrix == 0., axis=1)]
    
    return matrix

def get_solution_matrix(solutions_df, problem):
    users = get_users_that_solved_the_problem(solutions_df, problem)

    solutions = []
    min_problem = float('inf')
    max_problem = float('-inf')

    for user in users:
        past_problems = get_past_problems(solutions_df, user, problem)

        solutions.append(past_problems)

        min_problem = min([min_problem] + past_problems)
        max_problem = max([max_problem] + past_problems)

    nr_problems = max_problem - min_problem + 1
    nr_users = len(users)
    matrix = np.zeros((nr_problems, nr_users))

    for user in range(nr_users):
        for problem in solutions[user]:
            problem_index = problem - min_problem
            matrix[problem_index][user] = 1
            
    return optimize_matrix(matrix)

Com a matriz $M$, podemos executar o nosso algoritmo de otimização e então obter os valores de $X$ e $\theta$.

Nessa situação, podemos fazer a seguinte leitura:

> $X_{i}$ representa os requisitos do problema $P_i$ enquanto que $\theta_j$ representa as habilidades do usuário $U_j$.

Como anteriormente discutido, acreditamos que todos os problemas resolvidos antes de $P$ por um usuário $U$, contribuem para o desenvolvimento de suas habilidades, desta forma podemos concluir que o conjunto de habilidades $\theta$ ou o conjunto de requisitos $X$ formam as características do problema $P$.

Atribuímos então a $\theta_P$ a média de $\theta$.



In [17]:
def get_theta_of_problem(matrix):
    nr_problems, nr_users = matrix.shape

    theta_users = np.random.rand(nr_users, NR_FEATURES)
    theta_problems = np.zeros((nr_problems, NR_FEATURES))

    matrix_t = np.transpose(matrix)

    i = 0
    while i <= NR_EPOCHS:
        clear_output(wait=True)
        print('Progress: {}%'.format(100 * i/NR_EPOCHS))
        i += 1

        for problem in range(nr_problems):
            x = np.array(
                theta_users
            )

            y = np.array(
                matrix[problem]
            )

            theta_problems[problem] = gradient_descent(x, y)[1:]

        for user in range(nr_users):
            x = np.array(
                theta_problems
            )

            y = np.array(
                matrix_t[user]
            )

            theta_users[user] = gradient_descent(x, y)[1:]
    
    return theta_problems.mean(axis=0)

## As características do usuário

Com as características dos problemas do nosso dataset já definidas, podemos então encontrar as características do nosso usuário. Para isso, iremos construir a mesma matriz $M$, porém utilizando todo o histórico do usuário em questão.

In [18]:
def get_theta_of_user(problems_solved, thetas_solved):
    
    if not problems_solved:
        return np.zeros(NR_FEATURES)

    nr_problems = len(problems_solved)
    
    y = np.ones(nr_problems)
    x = np.array(list(thetas_solved.values()))

    thetas = [thetas_solved[t] for t in thetas_solved]    
    theta_of_user = np.max(thetas, axis=0)

    return theta_of_user

## Recomendação

Neste momento, temos para cada problema do nosso dataset suas características, e para cada usuário a capacidade de calcular suas habilidades baseado em seu histórico. Com isso, podemos então encontrar o melhor problema para o seu treinamento.

A recomendação é feita da seguinte forma:

- É retirado todos os problemas que possuem todos os requisitos inferiores as habilidades do usuário.
- É selecionado então o problema mais próximo a habilidade do usuário
- A distância é calculada através da distância do vetor de habilidades e o vetor de requisitos.

In [19]:
def find_closest_problem(theta_user, theta_problems):
    problems = [problem for problem in theta_problems if (theta_problems[problem] > theta_user).all()]
    points = [theta_problems[problem] for problem in problems]
    
    if not points:
        return None
    
    tree = spatial.KDTree(points)
    distance, index = tree.query(theta_user)
    
    return problems[index]
    
def get_next_problem(problems_solved, thetas):
    thetas_solved = {}
    thetas_not_solved = {}
    
    for problem in problems_solved:
        thetas_solved[problem] = thetas[problem]
    
    for theta in thetas:
        if theta not in problems_solved:
            thetas_not_solved[theta] = thetas[theta]
            
    if not thetas_not_solved:
        return None
    
    theta_user = get_theta_of_user(problems_solved, thetas_solved)
    problem = find_closest_problem(theta_user, thetas_not_solved)
    
    return problem


In [20]:
def get_thetas(solutions_df):
    thetas = {}

    for problem in get_problems(solutions_df):
        matrix = get_solution_matrix(solutions_df, problem)
        theta = get_theta_of_problem(matrix)
        thetas[problem] = theta

    return thetas

In [21]:
def simulate_user(thetas):
    problems_solved = []
    while True:
        next_problem = get_next_problem(problems_solved, thetas)

        if not next_problem:
            break

        problems_solved.append(next_problem)
        
    return problems_solved

In [22]:
# refactor
def add_theta_to_dataset():
    problems_df = pd.read_csv('problems_uva.csv')
    problems_df = problems_df[problems_df['problem_id'].isin(problems_solved)]

    # add theta column
    for problem_id in problems_solved:
        problems_df.loc[problems_df['problem_id'] == problem_id, 'theta'] = thetas[problem_id].sum()

    problems_df = problems_df.sort_values(["theta"])

    problems_df

In [23]:
def save_result(problems_solved):
    problems_uva_df = pd.read_csv('problems_uva.csv')
    problems_uva_df = problems_uva_df[problems_uva_df['category_id'] == CATEGORY]
    problems_suggested_df = problems_uva_df[problems_uva_df['problem_id'].isin(problems_solved)]

    result = {
        'nr_problems': len(problems_uva_df),
        'nr_problems_not_starred': len(problems_uva_df[problems_uva_df['starred'] == False]),
        'nr_problems_starred': len(problems_uva_df[problems_uva_df['starred'] == True]),

        'nr_problems_suggested': len(problems_solved),
        'nr_problems_suggested_not_starred': len(problems_suggested_df[problems_suggested_df['starred'] == False]),
        'nr_problems_suggested_starred': len(problems_suggested_df[problems_suggested_df['starred'] == True]),

        'problems_suggested': problems_solved,
        'category': CATEGORY,
        'features': NR_FEATURES,
        'epochs': NR_EPOCHS,
        'iterations': NR_ITERATIONS,
    }

    filename = 'results/C{}-F{}-{}E-{}I.npy'.format(CATEGORY, NR_FEATURES, NR_EPOCHS, NR_ITERATIONS)
    np.save(filename, result)
    
    problems_suggested_df

In [ ]:
def load_result(filename):
    data = np.load(filename).item()
    return data

In [ ]:
for i in range(1, 9): # category
    for j in range(100, 600, 100): # nr_iterations
        for k in range(1, 20): # epochs
            for l in range(1, 20): #features
                NR_FEATURES = l
                NR_EPOCHS = k
                NR_ITERATIONS = j
                CATEGORY = 7 # i
                
                solutions_df = read_dataset()
                thetas = get_thetas(solutions_df)
                problems_solved = simulate_user(thetas)
                save_result(problems_solved)

Progress: 100.0%


In [ ]:
#desenvolver gráfico com level, solucoes, id, submissoes
#verificar se a sequencia com 10 epocas esta dentro das sugestoes com 16 epocas (4,8,16,32)